In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer

In [2]:
data = pd.read_csv('../../../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
data['churn'] = data.Churn.map({'No':0, 'Yes':1})
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1


In [10]:
data.Churn.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

ValueError: could not convert string to float: ''

In [21]:
X = data.drop(['Churn', 'churn', 'customerID'], axis=1)
y = data['churn']

X_t, X_test, y_t, y_test = train_test_split(X, y, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_t, y_t, random_state=10)

In [22]:
cat_columns = list(X.select_dtypes('object').columns)
cat_columns

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'TotalCharges']

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [24]:
ohe = OneHotEncoder()

In [25]:
ohe.fit(X_train[cat_columns])

OneHotEncoder()

In [26]:
X_train_encoded = ohe.transform(X_train[cat_columns])
X_val_encoded = ohe.transform(X_val[cat_columns])

ValueError: Found unknown categories ['7365.7', '1905.4', '6555.2', '3930.55', '264.85', '1476.25', '7737.55', '755.6', '5536.5', '313.6', '4968', '7397', '1798.65', '333.6', '525.55', '74.4', '740.55', '2043.45', '7222.75', '211.95', '2773.9', '7299.65', '1375.6', '3019.7', '1375.15', '5848.6', '1923.5', '4549.45', '6589.6', '5817', '250.05', '1376.5', '116.95', '7244.7', '580.8', '4250.1', '889', '256.25', '3973.2', '220.65', '226.55', '6876.05', '836.35', '49.3', '6856.95', '7852.4', '4932.5', '1685.9', '4146.05', '5971.25', '272.95', '2879.2', '2416.1', '6014.85', '577.15', '197.4', '3810.55', '5232.9', '4534.45', '711.9', '2107.15', '854.8', '4615.9', '2021.35', '355.2', '2861.45', '810.3', '490.65', '1412.4', '4917.75', '4036.85', '4657.95', '961.4', '1144.5', '3886.85', '4807.35', '188.1', '325.45', '384.5', '4523.25', '161.65', '143.35', '2647.2', '4375.8', '5956.85', '1423.85', '178.85', '5375.15', '1389.85', '3958.2', '5552.05', '2000.2', '1094.35', '855.1', '105.5', '780.5',

In [30]:
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No,0
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No,0
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No,0
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes,1


In [40]:
data.loc[data['churn']==1]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,churn
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1
13,0280-XJGEX,Male,0,No,No,49,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7021,1699-HPSBG,Male,0,No,No,12,Yes,No,DSL,No,...,Yes,Yes,No,One year,Yes,Electronic check,59.80,727.8,Yes,1
7026,8775-CEBBJ,Female,0,No,No,9,Yes,No,DSL,No,...,No,No,No,Month-to-month,Yes,Bank transfer (automatic),44.20,403.35,Yes,1
7032,6894-LFHLY,Male,1,No,No,1,Yes,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,75.75,75.75,Yes,1
7034,0639-TSIQW,Female,0,No,No,67,Yes,Yes,Fiber optic,Yes,...,No,Yes,No,Month-to-month,Yes,Credit card (automatic),102.95,6886.25,Yes,1


In [31]:
1142/7043

0.1621468124378816

In [42]:
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No,0
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No,0
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No,0
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes,1


In [44]:
data.corr()

,SeniorCitizen,tenure,MonthlyCharges,churn
SeniorCitizen,1.000000,0.016567,0.220173,0.150889
tenure,0.016567,1.000000,0.247900,-0.352229
MonthlyCharges,0.220173,0.247900,1.000000,0.193356
churn,0.150889,-0.352229,0.193356,1.000000


In [45]:
X_train_sc_df = pd.read_csv('../Warren/X_train_sc.csv')
X_train_sc_df.head()

,gender,partner,dependents,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,seniorcitizen,tenure,monthlycharges,totalcharges
0,0,1,1,1,0,0,2,2,0,2,0,0,0,0,0,-0.427069,-0.017944,-0.107867,-0.183246
1,0,1,0,1,2,1,2,0,0,2,2,0,0,1,1,-0.427069,-0.875263,1.056297,-0.468087
2,0,1,1,1,0,2,1,1,1,1,1,1,2,0,1,-0.427069,1.165973,-1.501185,-0.428393
3,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,-0.427069,-1.038562,0.174811,-0.781296
4,0,0,1,1,0,2,1,1,1,1,1,1,1,0,1,-0.427069,0.676076,-1.511220,-0.601398


In [48]:
y_train_enc_df = pd.read_csv('../Warren/y_train_enc.csv')
y_train_enc_df.head()

,churn
0,0
1,1
2,0
3,0
4,0


In [50]:
X_and_y_train_df = pd.concat([X_train_sc_df, y_train_enc_df], axis=1)

In [53]:
X_and_y_train_df.corr().sort_values('churn')

,gender,partner,dependents,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,seniorcitizen,tenure,monthlycharges,totalcharges,churn
contract,-0.016255,0.290546,0.241531,-0.020188,0.107874,0.081580,0.365803,0.287436,0.348463,0.428114,0.198854,0.216370,1.000000,-0.176169,-0.220984,-0.140890,0.668540,-0.073143,0.446352,-0.405451
tenure,-0.007520,0.381973,0.153010,-0.001749,0.332390,-0.041575,0.315016,0.373978,0.355334,0.315305,0.276553,0.281734,0.668540,0.006452,-0.367005,0.018760,1.000000,0.247904,0.827485,-0.358398
onlinesecurity,-0.020516,0.145325,0.157988,-0.014071,-0.004261,-0.023282,1.000000,0.182703,0.168825,0.276620,0.023402,0.035753,0.365803,-0.164518,-0.086876,-0.129141,0.315016,-0.059030,0.244852,-0.291231
techsupport,-0.016404,0.103605,0.132562,-0.021445,0.007628,-0.031707,0.276620,0.204859,0.236417,1.000000,0.134233,0.145292,0.428114,-0.112696,-0.106037,-0.159099,0.315305,-0.025153,0.265912,-0.288184
onlinebackup,-0.019386,0.145162,0.101487,0.027518,0.117249,0.028697,0.182703,1.000000,0.196503,0.204859,0.136926,0.121973,0.287436,-0.014030,-0.117107,-0.022804,0.373978,0.109867,0.376346,-0.219318
totalcharges,-0.012138,0.317613,0.061747,0.106943,0.441912,-0.179339,0.244852,0.376346,0.372641,0.265912,0.377512,0.379568,0.446352,0.158375,-0.328446,0.100220,0.827485,0.649257,1.000000,-0.202782
deviceprotection,-0.010192,0.156635,0.081873,-0.008250,0.124389,0.043852,0.168825,0.196503,1.000000,0.236417,0.270676,0.293376,0.348463,-0.049897,-0.136896,-0.028190,0.355334,0.156403,0.372641,-0.185515
dependents,0.008427,0.444457,1.000000,-0.014330,-0.022439,0.028112,0.157988,0.101487,0.081873,0.132562,0.034963,0.020672,0.241531,-0.104829,-0.052242,-0.208476,0.153010,-0.103250,0.061747,-0.172287
partner,-0.007736,1.000000,0.444457,0.007396,0.144504,-0.019289,0.145325,0.145162,0.156635,0.103605,0.130794,0.121959,0.290546,-0.013878,-0.137626,0.005567,0.381973,0.106316,0.317613,-0.146286
internetservice,-0.017792,-0.019289,0.028112,0.385418,-0.098530,1.000000,-0.023282,0.028697,0.043852,-0.031707,0.104269,0.100662,0.081580,-0.130280,0.090197,-0.031819,-0.041575,-0.326385,-0.179339,-0.039844


In [54]:
X_and_y_train_df

,gender,partner,dependents,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,seniorcitizen,tenure,monthlycharges,totalcharges,churn
0,0,1,1,1,0,0,2,2,0,2,0,0,0,0,0,-0.427069,-0.017944,-0.107867,-0.183246,0
1,0,1,0,1,2,1,2,0,0,2,2,0,0,1,1,-0.427069,-0.875263,1.056297,-0.468087,1
2,0,1,1,1,0,2,1,1,1,1,1,1,2,0,1,-0.427069,1.165973,-1.501185,-0.428393,0
3,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,-0.427069,-1.038562,0.174811,-0.781296,0
4,0,0,1,1,0,2,1,1,1,1,1,1,1,0,1,-0.427069,0.676076,-1.511220,-0.601398,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,-0.427069,0.716901,-0.663187,-0.020924,0
3957,1,1,0,1,2,1,2,2,2,0,2,2,1,1,2,-0.427069,1.574220,1.504567,2.471672,0
3958,0,0,0,1,2,0,2,0,2,2,2,2,1,1,1,-0.427069,0.104530,0.713404,0.368814,1
3959,0,1,1,1,0,0,2,2,0,0,2,2,0,0,3,-0.427069,-0.630315,0.332040,-0.470857,0
